In [1]:
import pandas as pd
import pyarrow.parquet as pq
import numpy as np
import datetime
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import pickle

#### Q6. Evaluating the model
Now let's apply this model to the validation dataset (Feb 2021).

    What's the RMSE on validation?

     - 6.01
     - 11.01
     - 16.01
     - 21.01


In [2]:
with open( "dict_vectorizer.p", "rb" ) as f:
    dv = pickle.load(f)

In [3]:
with open( "model.p", "rb" ) as f:
    lr = pickle.load(f)

In [4]:
df_feb = pq.read_table(source='fhv_tripdata_2021-02.parquet').to_pandas()

In [5]:
df_feb.head()

,dispatching_base_num,pickup_datetime,dropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Affiliated_base_number
0,B00013,2021-02-01 00:01:00,2021-02-01 01:33:00,NaN,NaN,None,B00014
1,B00021,2021-02-01 00:55:40,2021-02-01 01:06:20,173.0,82.0,None,B00021
2,B00021,2021-02-01 00:14:03,2021-02-01 00:28:37,173.0,56.0,None,B00021
3,B00021,2021-02-01 00:27:48,2021-02-01 00:35:45,82.0,129.0,None,B00021
4,B00037,2021-02-01 00:12:50,2021-02-01 00:26:38,NaN,225.0,None,B00037


In [6]:
def data_prep(df):
    df['duration'] = df['dropOff_datetime'] -  df['pickup_datetime']
    df['duration'] = df['duration'].apply(lambda td: td.total_seconds()/60)
    df = df[(df.duration>=1) & (df.duration<=60)]
    
    df['PUlocationID'] = df['PUlocationID'].fillna(-1)
    df['DOlocationID'] = df['DOlocationID'].fillna(-1)
    
    categorical = ['PUlocationID', 'DOlocationID']
    df[categorical] = df[categorical].astype(str)
    
    train_dict = df[categorical].to_dict(orient = 'records')
    
    X= dv.transform(train_dict)
    y = df['duration']
    
    return X,y

In [7]:
X_val,y_val = data_prep(df_feb)

C:\Users\mayan\anaconda3\envs\gputest\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\mayan\anaconda3\envs\gputest\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\mayan\anaconda3\envs\gputest\lib\site-packages\pandas\core\frame.py:3069: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [8]:
print(X_val.shape)
print(y_val.shape)

(990113, 525)
(990113,)


In [9]:
y_val_pred = lr.predict(X_val)
mse  = mean_squared_error(y_val, y_val_pred)
print(mse**.5)

11.01428321783417
